## Working with TiTiler-EoPF


#### Requirements
- folium
- httpx

`pip install folium httpx`

In [ ]:
# Uncomment this line if you need to install the dependencies
# !python -m pip install folium httpx

In [ ]:
import json
import httpx
from folium import Map, TileLayer

from IPython.display import Image

%matplotlib inline

In [ ]:
titiler_endpoint = "http://127.0.0.1:8000"

### Conformances

In [ ]:
r = httpx.get(f"{titiler_endpoint}/conformance").json()
print(json.dumps(r, indent=4))

In [ ]:
collection_id = "sentinel-2-l2a"
item_id = "S2A_MSIL2A_20250918T100041_N0511_R122_T33TUG_20250918T121119"

## Variables

In [ ]:
# Fetch Metadata for all available variables
r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/dataset/keys",
    timeout=20,
).json()

print("list of supported variables: ")
print(json.dumps(list(r), indent=4))

### Info

In [ ]:
# Fetch Metadata for all available variables
r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/info",
    timeout=10,
).json()

print(json.dumps(r, indent=4))

In [ ]:
# Fetch Metadata for a specific Variable
r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/info",
    params={
        "variables": "/measurements/reflectance/r60m:b05",
    },
    timeout=10,
).json()

print(json.dumps(r, indent=4))

## Display tiles

### Single Variable

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/WebMercatorQuad/tilejson.json",
    params={
        "variables": "/quality/l2a_quicklook/r10m:tci",
    },
    timeout=10,
).json()
print(r)
bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2), zoom_start=8
)

TileLayer(tiles=r["tiles"][0], opacity=1, attr="ESA EoPF").add_to(m)
m

### Band Combination: False Color Nir/Red/Green

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/WebMercatorQuad/tilejson.json",
    params=[
        ("variables", "/measurements/reflectance/r10m:b08"),
        ("variables", "/measurements/reflectance/r10m:b04"),
        ("variables", "/measurements/reflectance/r10m:b03"),
        ("rescale", "0,1"),
    ],
).json()
print(r)
bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2), zoom_start=9
)

TileLayer(tiles=r["tiles"][0], opacity=1, attr="ESA EoPF").add_to(m)

m

### Band Math: NDVI

In [ ]:
red = "/measurements/reflectance/r10m:b04"
nir = "/measurements/reflectance/r10m:b08"

r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/WebMercatorQuad/tilejson.json",
    params=[
        ("expression", f"({nir}-{red})/({nir}+{red})"),
        ("rescale", "-1,1"),
        ("colormap_name", "viridis"),
    ],
    timeout=10,
).json()
print(r)
bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2), zoom_start=9
)

TileLayer(tiles=r["tiles"][0], opacity=1, attr="ESA EoPF").add_to(m)

m

### Preview

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/preview.png",
    params=[
        ("variables", "/measurements/reflectance/r10m:b08"),
        ("variables", "/measurements/reflectance/r10m:b04"),
        ("variables", "/measurements/reflectance/r10m:b03"),
        ("rescale", "0,1"),
        ("max_size", 256),
    ],
)
print(r.headers)
Image(r.content)

### OGC Maps

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/map",
    params=[
        ("variables", "/measurements/reflectance/r10m:b08"),
        ("variables", "/measurements/reflectance/r10m:b04"),
        ("variables", "/measurements/reflectance/r10m:b03"),
        ("rescale", "0,1"),
        ("width", 256),
        ("height", 256),
        ("f", "png"),
    ],
)
print(r.headers)
Image(r.content)

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/map",
    params=[
        ("variables", "/measurements/reflectance/r10m:b08"),
        ("variables", "/measurements/reflectance/r10m:b04"),
        ("variables", "/measurements/reflectance/r10m:b03"),
        ("rescale", "0,1"),
        ("width", 256),
        ("height", 256),
        ("f", "png"),
        ("crs", "EPSG:32633"),
    ],
)
print(r.headers)
Image(r.content)

### Part

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/bbox/12.876475023062739,41.85068925356174,13.112219919388622,42.044510255080894.png",
    params=[
        ("variables", "/measurements/reflectance/r10m:b08"),
        ("variables", "/measurements/reflectance/r10m:b04"),
        ("variables", "/measurements/reflectance/r10m:b03"),
        ("rescale", "0,1"),
        ("width", 256),
        ("height", 256),
    ],
)
print(r.headers)
Image(r.content)

In [ ]:
import json

feat = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                [13.007976382149138, 42.044510255080894],
                [12.876475023062739, 41.920657412019864],
                [12.986959890789478, 41.85068925356174],
                [13.059243278886385, 41.85491287703039],
                [13.10232704916828, 41.90887095414385],
                [13.112219919388622, 41.97901081115802],
                [13.084478709917647, 42.03192521656223],
                [13.007976382149138, 42.044510255080894],
            ]
        ],
        "type": "Polygon",
    },
}

r = httpx.post(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/feature.png",
    data=json.dumps(feat),
    params=[
        ("variables", "/measurements/reflectance/r10m:b08"),
        ("variables", "/measurements/reflectance/r10m:b04"),
        ("variables", "/measurements/reflectance/r10m:b03"),
        ("rescale", "0,1"),
        ("width", 256),
        ("height", 256),
    ],
)
print(r.headers)
Image(r.content)